# Action Sequence Graph Tutorial

This tutorial covers use of the ActionArchitecture in the Function class, which is useful for representing a Function's progress through an action sequence graph, the different actions/tasks to be performed by a function (e.g., modes of operation, etc)..

```
Copyright © 2024, United States Government, as represented by the Administrator of the National Aeronautics and Space Administration. All rights reserved.

The “"Fault Model Design tools - fmdtools version 2"” software is licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

In [1]:
import fmdtools.sim.propagate as prop
import inspect

Action architectures are used within a function block to represent Action Sequence Graphs - actions that the function performs and their sequence. Actions in an ActionArchitecture are respresented by `Action` blocks, which are similar to function and component blocks in that they have:
- flow connections
- modes, and
- behaviors

Flow connections are routed to the action in the function block definition and represent the *shared variables* between the actions.

Modes are similar to function modes and are instantiated (as they are in components) at both the Function and Action level. Using the `name=` option enables one to tag these modes as action modes at the function level while using the same local name

Below we define the states to performed by the ASG:
- Outcome, which tracks the number of actions and perceptions performed by the ASG
- HazardState, which specifies whether a hazard is present, percieved, or mitigated

In [2]:
from demo_model import OutcomeStates, Outcome, HazardState, Hazard
print(inspect.getsource(OutcomeStates))
print(inspect.getsource(Outcome))
print(inspect.getsource(HazardState))
print(inspect.getsource(Hazard))

ModuleNotFoundError: No module named 'demo_model'

Below we define three actions for use in a given model:
- Perceive, a user's perception abilities/behaviors. In this function the user percieves a hazard (unless their perception fails)
- Act, the user's action which they perform to mitigate the hazard.
- Done, the user's state when they are done performing the action.

In [3]:
from demo_model import Perceive, Act, Done
print(inspect.getsource(Perceive))
print(inspect.getsource(Act))
print(inspect.getsource(Done))

To proceed through the sequence of actions, *conditions* must be met between each action. In these actions, we have defined the following conditions:
- Percieve.percieved: perception is done if the hazard is percieved
- Act.acted: the action is complete if the action was performed
- Done.complete: the hazard mitigation is over (and mitigated state is reset to False)

To create the overall ASG structure, the following adds the flows, actions, and conditions to the ActionArchitecture class

In [4]:
from demo_model import Human
print(inspect.getsource(Human))

Note the use of the following methods:
- add_flow adds a flow so it can pass variables between actions. Here *Outcome* is an internal flow, while *Hazard* is an external flow.

In [5]:
from fmdtools.define.architecture.action import ActionArchitecture
help(ActionArchitecture.add_flow)

- add_act adds the action to the function and hands it the given flows and parameters. Here the actions are "Percieve", "Act", and "Done"

In [6]:
help(ActionArchitecture.add_act)

- add_cond specifies the conditions for going from one action to another.

In [7]:
help(ActionArchitecture.add_cond)

- ASG.build finally constructs the structure of the ASG (see: self.action_graph and self.flow_graph) and determines the settings for the simulation. In DetectHazard, default options are used, with the first action specified as "Percieve" and also with it specified that the actions propagate in the dynamic step (rather than static step)

In [8]:
help(ActionArchitecture.build)

We can look at the attributes of the `ASG` by instantiating it:

In [9]:
h = Human()
h

`ActionArchitectureGraph` can also be used to visualize the flow of actions (e.g., which are active, flows vs conditions, etc.).

This graph can also be generated using `ActionArchitecture.as_modelgraph()`

In [10]:
from fmdtools.define.architecture.action import ActionArchitectureGraph

In [11]:
ag = ActionArchitectureGraph(h)

In [12]:
fig, ax = ag.draw()

As shown, the "Percieve" action is active (green), while the inactive actions are shown in blue. This action is active because it was defined as initial_action in the ASG definition.

These are stored as attributes in the underlying graph structure:

In [13]:
ag.g.nodes['perceive']

### Function-level Simulation

`ActionArchitecture`s must be instantiated within a `Function` in order to be simulable with methods in `propagate`. Below we instantiate a `Function` and show how an `ActionArchitecture` can be simulated independent of other Model attributes. 

In [14]:
from demo_model import DetectHazard
print(inspect.getsource(DetectHazard))

If we update the action, we can see the ASG progress between states:

In [15]:
ex_fxn = DetectHazard('detect_hazard')
ex_fxn.t.dt=1.0

ex_fxn.aa.flows['hazard']

In [16]:
ex_fxn.aa.flows['hazard'].s.present=True
ex_fxn('dynamic', time= 1)

In [17]:
ag = ActionArchitectureGraph(ex_fxn.aa)
ag.draw()

In [18]:
ex_fxn.aa.flows['hazard']

In [19]:
assert ex_fxn.aa.flows['hazard'].s.present

In [20]:
ex_fxn.aa.flows['outcome']

In [21]:
assert ex_fxn.aa.flows['outcome'].s.num_perceptions==1
assert ex_fxn.aa.flows['outcome'].s.num_actions==1

As shown, each of the actions are progressed throuh in a single timestep until the ASG is in the "Done" action

In [22]:
ex_fxn.aa.flows['hazard'].s.present=False
ex_fxn('dynamic', time= 2)

In [23]:
ag = ActionArchitectureGraph(ex_fxn.aa)
fig, ax = ag.draw()

As shown, now that the hazard is no longer present, the "Ready" Condition is triggered and the ASG goes back to the percieve state.

In [24]:
ex_fxn.aa.flows['outcome']

In [25]:
ex_fxn.aa.flows['hazard']

In [26]:
ex_fxn.hazard

In [27]:
assert ex_fxn.aa.flows['hazard'].s.present==False
assert ex_fxn.aa.flows['hazard'].s.mitigated==False
assert id(ex_fxn.hazard) == id(ex_fxn.aa.flows['hazard'])

This is essentially what a step-though of a simulation looks like. In practice, we simulate these behaviors using `propagate` as long as the `ActionArchitecture` is contained in a `Function`. Here we use the `disturbances` argument to simulate an external hazard occuring at `t=5`:

In [28]:
result_indiv, hist_indiv = prop.nominal(ex_fxn, disturbances={5:{'aa.flows.hazard.s.present':True}})

In [29]:
fig, axs = hist_indiv.plot_line('aa.flows.hazard.s.present',
                                'aa.flows.hazard.s.percieved',
                                'aa.flows.hazard.s.mitigated', figsize=(10,5))

In [30]:
hist_indiv.aa.active_actions

In [31]:
assert hist_indiv.aa.flows.hazard.s.mitigated[5]
assert hist_indiv.aa.active_actions[5]=={'done'}

### Model Simulation

Below, this function is placed in the context of a FunctionalArchitecture model so we can see how it behaves in the context of its interactions.

In [32]:
from demo_model import ProduceHazard, PassStates, PassHazard, HazardModel
print(inspect.getsource(ProduceHazard))
print(inspect.getsource(PassStates))
print(inspect.getsource(PassHazard))
print(inspect.getsource(HazardModel))

As shown, this model connects the `detect_hazard` with a `detect_hazard` function, which are used (1) to load the system and then to (2) tabulate the number of mitigations.

Below we show how this translates into simulation results:

In [33]:
mdl = HazardModel()
endstate,  mdlhist = prop.nominal(mdl)

Below we look at the states of the functions/flows to see how this has simulated.

In [34]:
mdlhist

In [35]:
restab = mdlhist.get('fxns.detect_hazard.aa.active_actions', 'flows.hazard.s').as_table()

In [36]:
restab

As shown, the ASG alternates between Perceive (when the hazard is not present) and Done (when the hazard is present). As a result, all of the present hazards (above) are also perceived and mitigated, and no hazards are propagated.

Or, in plot form:

In [37]:
fig, axs = mdlhist.plot_line('fxns.pass_hazard.s.hazards_mitigated',
                                'fxns.pass_hazard.s.hazards_propagated',
                                'flows.hazard.s.present',
                                'flows.hazard.s.percieved',
                                'flows.hazard.s.mitigated', figsize=(10,5))

As shown, perceptions and actions track the hazards mitigated.

### Fault Simulation
Below we will simulate a fault and see how it tracks in the model.

In [38]:
result_fault, mdlhist_fault = prop.one_fault(mdl, 'detect_hazard','perceive_failed', time=4, desired_result='graph')

In [39]:
mdlhist_fault.faulty.fxns.detect_hazard.aa.acts.act

As shown, this fault results in the hazard not being perceived (and thus the hazard propagating)

In [40]:
fig, axs = mdlhist_fault.plot_line('fxns.pass_hazard.s.hazards_mitigated',
                                'fxns.pass_hazard.s.hazards_propagated',
                                'flows.hazard.s.present',
                                'flows.hazard.s.percieved',
                                'flows.hazard.s.mitigated', figsize=(10,5), time_slice=[4])

As shown, this only shows up in the PassHazard function (since the fault is removed in one timestep).

In [41]:
result_fault.graph.draw()

To see this in more detail, we will process the results history and then use `graph.results_from` at the time of the fault. Note how the model histories can be processed below:

In [42]:
reshist = mdlhist_fault.get_degraded_hist(*mdl.fxns, *mdl.flows)
reshist.as_table()

In [43]:
reshist = mdlhist_fault.get_faulty_hist(*mdl.fxns, *mdl.flows)
reshist.as_table()

In [44]:
mdlhist_fault.faulty.fxns.detect_hazard

In [45]:
mdl.fxns['detect_hazard'].h

We can further use `draw_from` to view the state of the ASG. See below:

In [46]:
result_fault.graph.draw_from(4, mdlhist_fault)

Note the lack of a fault at this time-step, despite it being instantiated here. This is because the fault was removed at the end of the same time-step it was added in.

The 'unable' fault, on the other hand, stays throughout the simulation and thus shows up:

In [47]:
endstate_unable, mdlhist_unable = prop.one_fault(mdl, 'detect_hazard','act_unable', time=4, 
                                                 desired_result={'graph','graph.fxns.detect_hazard.aa'})

fig, axs = mdlhist_unable.plot_line('fxns.pass_hazard.s.hazards_mitigated',
                                'fxns.pass_hazard.s.hazards_propagated',
                                'flows.hazard.s.present',
                                'flows.hazard.s.percieved',
                                'flows.hazard.s.mitigated',
                                'fxns.detect_hazard.aa.flows.hazard.s.percieved',
                                'fxns.detect_hazard.aa.acts.act.m.faults.unable',
                            figsize=(10, 5), time_slice=[4])

In [48]:
mdlhist_unable.faulty.fxns.detect_hazard.aa.active_actions

In [49]:
mdl.fxns['detect_hazard'].h.aa.acts.perceive

In [50]:
mdl.fxns['detect_hazard'].aa.create_hist([0.0])

In [51]:
mdl.fxns['detect_hazard'].aa.track

In [52]:
mdl.fxns['detect_hazard'].aa.h.acts.perceive.m.faults.failed

In [53]:
mdlhist_unable.nominal.fxns.detect_hazard.aa.acts

In [54]:
endstate_unable.graph.draw_from(4, mdlhist_unable)

In [55]:
fig, ax = endstate_unable['graph.fxns.detect_hazard.aa'].draw_from(0, mdlhist_unable)

In [56]:
fig, ax = endstate_unable['graph.fxns.detect_hazard.aa'].draw_from(5, mdlhist_unable, rem_ind=0)